In [6]:
import os, random
from pathlib import Path
from collections import Counter
import pandas as pd

from dotenv import load_dotenv
from neo4j import GraphDatabase

In [ ]:
# Load .env from current folder
load_dotenv(Path(".env"), override=True)

def _strip_quotes(v):
    return None if v is None else v.strip().strip('"').strip("'")

URI = _strip_quotes(os.getenv("NEO4J_URI"))
USER = _strip_quotes(os.getenv("NEO4J_USERNAME"))
PWD  = _strip_quotes(os.getenv("NEO4J_PASSWORD"))

GT_DB = _strip_quotes(os.getenv("NEO4J_CONSTRAINT_DB")) or "restaurants-gt"
INSTANCE_DB = _strip_quotes(os.getenv("NEO4J_INSTANCE_DB")) or "restaurants-instance"

AUTH = (USER, PWD)

print("URI:", URI)
print("GT_DB:", GT_DB)
print("INSTANCE_DB:", INSTANCE_DB)


In [7]:
# Load .env from current folder
load_dotenv(Path(".env"), override=True)

def _strip_quotes(v):
    return None if v is None else v.strip().strip('"').strip("'")

URI = _strip_quotes(os.getenv("NEO4J_URI"))
USER = _strip_quotes(os.getenv("NEO4J_USERNAME"))
PWD  = _strip_quotes(os.getenv("NEO4J_PASSWORD"))

GT_DB = _strip_quotes(os.getenv("NEO4J_CONSTRAINT_DB")) or "restaurants-gt"
INSTANCE_DB = _strip_quotes(os.getenv("NEO4J_INSTANCE_DB")) or "restaurants-instance"

AUTH = (USER, PWD)

print("URI:", URI)
print("GT_DB:", GT_DB)
print("INSTANCE_DB:", INSTANCE_DB)


URI: neo4j://127.0.0.1:7687
GT_DB: restaurant-constraint
INSTANCE_DB: restaurant-instance


In [8]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connected")

    # APOC check
    rec = driver.execute_query("RETURN apoc.version() AS v", database_=GT_DB).records[0]
    print("APOC version:", rec["v"])


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [17]:
from pathlib import Path

CLEAN_DIR = Path("datasets/temp")  # ← change ONLY this if needed

def get_latest_file(directory: Path, prefix: str):
    """
    Return the most recently modified file in `directory`
    whose name starts with `prefix`.
    """
    candidates = list(directory.glob(f"{prefix}*"))
    if not candidates:
        raise FileNotFoundError(
            f"No file found in {directory} with prefix '{prefix}'"
        )
    # pick most recently modified
    return max(candidates, key=lambda p: p.stat().st_mtime)

# Auto-detect cleaned artifacts
CLEAN_RESTAURANTS_PATH = get_latest_file(CLEAN_DIR, "restaurants_cleaned_")
CLEAN_EDGES_PATH       = get_latest_file(CLEAN_DIR, "restaurant_similarities_cleaned_")

print("Using cleaned restaurants file:", CLEAN_RESTAURANTS_PATH)
print("Using cleaned edges file:", CLEAN_EDGES_PATH)


Using cleaned restaurants file: datasets\temp\restaurants_cleaned_20260206-195400.txt
Using cleaned edges file: datasets\temp\restaurant_similarities_cleaned_20260206-195400.txt


In [9]:
def load_clean_restaurants(path: Path):
    rows = []
    with open(path, encoding="utf-8") as f:
        header = next(f)  # id name area_code addr city
        for line in f:
            rid, name, area_code, addr, city = line.rstrip("\n").split("\t")
            rows.append({
                "id": str(rid),
                "name": name,
                "area_code": None if area_code in ("None","") else str(area_code),
                "addr": addr,
                "city": city
            })
    return rows

def load_clean_edges(path: Path):
    edges = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            s = line.strip().lstrip("(").rstrip(")")
            if not s:
                continue
            a, b = [x.strip() for x in s.split(",")]
            edges.append((str(a), str(b)))
    return edges

gt_restaurants = load_clean_restaurants(CLEAN_RESTAURANTS_PATH)
gt_edges = load_clean_edges(CLEAN_EDGES_PATH)

print("restaurants:", len(gt_restaurants))
print("edges:", len(gt_edges))
print("sample restaurant:", gt_restaurants[0])
print("sample edge:", gt_edges[0])


NameError: name 'CLEAN_RESTAURANTS_PATH' is not defined

In [10]:
def clear_database(driver, db):
    driver.execute_query("MATCH (n) DETACH DELETE n", database_=db)

def setup_constraints(driver, db):
    driver.execute_query("""
        CREATE CONSTRAINT restaurant_id_unique IF NOT EXISTS
        FOR (r:Restaurant) REQUIRE r.id IS UNIQUE
    """, database_=db)
    driver.execute_query("""
        CREATE CONSTRAINT areacode_code_unique IF NOT EXISTS
        FOR (a:AreaCode) REQUIRE a.code IS UNIQUE
    """, database_=db)

def import_restaurants_and_edges(driver, db, restaurants_list, edges_list):
    # Restaurants
    driver.execute_query("""
        UNWIND $restaurants AS r
        MERGE (n:Restaurant {id: r.id})
        SET n.name = r.name,
            n.addr = r.addr,
            n.city = r.city,
            n.area_code = r.area_code,
            n.label = r.area_code
    """, restaurants=restaurants_list, database_=db)

    # Undirected SIMILAR stored as 2 directed edges
    driver.execute_query("""
        UNWIND $pairs AS p
        MATCH (a:Restaurant {id: p[0]})
        MATCH (b:Restaurant {id: p[1]})
        MERGE (a)-[:SIMILAR]->(b)
        MERGE (b)-[:SIMILAR]->(a)
    """, pairs=edges_list, database_=db)

def build_constraint_graph_S(driver, db):
    # create AreaCode nodes from labels
    driver.execute_query("""
        MATCH (r:Restaurant)
        WITH DISTINCT r.label AS code
        WHERE code IS NOT NULL
        MERGE (:AreaCode {code: code})
    """, database_=db)
    # allow only same area_code (self-loop)
    driver.execute_query("""
        MATCH (a:AreaCode)
        MERGE (a)-[:ALLOWED]->(a)
    """, database_=db)

def count_violations(driver, db):
    rec = driver.execute_query("""
        MATCH (a:Restaurant)-[:SIMILAR]->(b:Restaurant)
        WHERE a.label IS NOT NULL AND b.label IS NOT NULL AND a.label <> b.label
        RETURN count(*) AS v
    """, database_=db).records[0]
    return int(rec["v"])

def fetch_labels(driver, db):
    res = driver.execute_query("""
        MATCH (r:Restaurant)
        RETURN r.id AS id, r.label AS label
    """, database_=db)
    return {str(r["id"]): str(r["label"]) for r in res.records}


In [11]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    # GT
    clear_database(driver, GT_DB)
    setup_constraints(driver, GT_DB)
    import_restaurants_and_edges(driver, GT_DB, gt_restaurants, gt_edges)
    build_constraint_graph_S(driver, GT_DB)
    print("GT violations:", count_violations(driver, GT_DB), "(expected 0)")

    # INSTANCE (clean copy)
    clear_database(driver, INSTANCE_DB)
    setup_constraints(driver, INSTANCE_DB)
    import_restaurants_and_edges(driver, INSTANCE_DB, gt_restaurants, gt_edges)
    build_constraint_graph_S(driver, INSTANCE_DB)
    print("INSTANCE violations:", count_violations(driver, INSTANCE_DB), "(expected 0)")


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [12]:
def fetch_restaurant_ids(driver, db):
    res = driver.execute_query("""
        MATCH (r:Restaurant)
        RETURN r.id AS id
        ORDER BY toInteger(r.id)
    """, database_=db)
    return [str(r["id"]) for r in res.records]

def fetch_area_codes(driver, db):
    res = driver.execute_query("""
        MATCH (a:AreaCode)
        RETURN a.code AS code
        ORDER BY a.code
    """, database_=db)
    return [str(r["code"]) for r in res.records]

def inject_label_noise(driver, db, k, seed=42):
    rng = random.Random(seed)
    ids = fetch_restaurant_ids(driver, db)
    codes = fetch_area_codes(driver, db)

    if len(codes) < 2:
        raise RuntimeError("Need at least 2 distinct area codes")

    k = min(k, len(ids))
    chosen_ids = rng.sample(ids, k)

    res = driver.execute_query("""
        UNWIND $ids AS id
        MATCH (r:Restaurant {id:id})
        RETURN r.id AS id, r.label AS old_label
    """, ids=chosen_ids, database_=db)

    updates = []
    for rec in res.records:
        rid = str(rec["id"])
        old = str(rec["old_label"])
        new = old
        while new == old:
            new = rng.choice(codes)
        updates.append({"id": rid, "old_label": old, "new_label": new})

    driver.execute_query("""
        UNWIND $updates AS u
        MATCH (r:Restaurant {id:u.id})
        SET r.noise_old_label = u.old_label,
            r.label = u.new_label,
            r.noise_type = "label_noise"
        SET r:Fraudulent
    """, updates=updates, database_=db)

    return updates


In [13]:
def evaluate_label_recovery(gt_labels, before_labels, after_labels):
    common = set(gt_labels) & set(before_labels) & set(after_labels)

    true_wrong = {rid for rid in common if before_labels[rid] != gt_labels[rid]}
    wrong_after = {rid for rid in common if after_labels[rid] != gt_labels[rid]}

    fixed = true_wrong - wrong_after
    broken = (common - true_wrong) & wrong_after

    TP = len(fixed)
    FN = len(true_wrong & wrong_after)
    FP = len(broken)

    precision = TP / (TP + FP) if (TP + FP) else 0.0
    recall    = TP / (TP + FN) if (TP + FN) else 0.0
    f1        = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0.0
    acc_after = (len(common) - len(wrong_after)) / len(common) if common else 0.0

    return {
        "nodes_compared": len(common),
        "true_wrong_before": len(true_wrong),
        "wrong_after": len(wrong_after),
        "fixed": len(fixed),
        "broken": len(broken),
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "label_accuracy_after": acc_after,
    }


In [14]:
def clear_repair_tags(driver, db):
    driver.execute_query("""
        MATCH (r:Restaurant)
        REMOVE r.repaired
        REMOVE r.repair_algo
        REMOVE r.repair_old_label
    """, database_=db)

APOC_GREEDY_QUERY = """
CALL apoc.periodic.commit("
  MATCH (r:Restaurant)-[:SIMILAR]->(n:Restaurant)
  WHERE r.label IS NOT NULL AND n.label IS NOT NULL AND r.label <> n.label
  WITH r, count(*) AS bad_degree
  ORDER BY bad_degree DESC, toInteger(r.id) ASC
  LIMIT 1
  WITH r
  WHERE r IS NOT NULL

  MATCH (r)-[:SIMILAR]->(m:Restaurant)
  WHERE m.label IS NOT NULL
  WITH r, m.label AS lab, count(*) AS c
  ORDER BY c DESC, lab ASC
  WITH r, collect({lab: lab, c: c}) AS counts
  WITH r,
       reduce(best={lab:r.label, c:0}, x IN counts |
         CASE WHEN x.c > best.c OR (x.c = best.c AND x.lab < best.lab)
              THEN x ELSE best END
       ) AS bestCand
  WITH r, bestCand.lab AS new_label
  WHERE new_label IS NOT NULL AND new_label <> r.label

  SET r.repair_old_label = r.label,
      r.label = new_label,
      r.repaired = true,
      r.repair_algo = 'apoc_greedy'

  RETURN 1 AS updates
", {batchSize: 1});
"""

def apoc_greedy_repair(driver, db):
    driver.execute_query(APOC_GREEDY_QUERY, database_=db)

def cost_num_relabels(driver, db, tag="apoc_greedy"):
    rec = driver.execute_query("""
        MATCH (r:Restaurant) WHERE r.repair_algo = $tag
        RETURN count(r) AS c
    """, tag=tag, database_=db).records[0]
    return int(rec["c"])


In [15]:
K = 10
SEED = 42

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    # reset INSTANCE to clean copy from GT payload
    clear_database(driver, INSTANCE_DB)
    setup_constraints(driver, INSTANCE_DB)
    import_restaurants_and_edges(driver, INSTANCE_DB, gt_restaurants, gt_edges)
    build_constraint_graph_S(driver, INSTANCE_DB)
    clear_repair_tags(driver, INSTANCE_DB)

    gt_labels = fetch_labels(driver, GT_DB)

    # noise
    inject_label_noise(driver, INSTANCE_DB, K, seed=SEED)
    before_labels = fetch_labels(driver, INSTANCE_DB)
    viol_before = count_violations(driver, INSTANCE_DB)

    # repair
    apoc_greedy_repair(driver, INSTANCE_DB)
    after_labels = fetch_labels(driver, INSTANCE_DB)
    viol_after = count_violations(driver, INSTANCE_DB)
    cost = cost_num_relabels(driver, INSTANCE_DB, tag="apoc_greedy")

    metrics = evaluate_label_recovery(gt_labels, before_labels, after_labels)

    summary = {
        "k_injected": K,
        "seed": SEED,
        "violations_before": viol_before,
        "violations_after": viol_after,
        "repair_cost_num_relabels": cost,
        **metrics
    }

summary


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [16]:
Ks = [5, 10, 20, 50]
SEED = 42

rows = []

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    gt_labels = fetch_labels(driver, GT_DB)

    for k in Ks:
        # reset INSTANCE to clean
        clear_database(driver, INSTANCE_DB)
        setup_constraints(driver, INSTANCE_DB)
        import_restaurants_and_edges(driver, INSTANCE_DB, gt_restaurants, gt_edges)
        build_constraint_graph_S(driver, INSTANCE_DB)
        clear_repair_tags(driver, INSTANCE_DB)

        # noise snapshots
        inject_label_noise(driver, INSTANCE_DB, k, seed=SEED)
        before_labels = fetch_labels(driver, INSTANCE_DB)
        viol_before = count_violations(driver, INSTANCE_DB)

        # apoc repair
        apoc_greedy_repair(driver, INSTANCE_DB)

        after_labels = fetch_labels(driver, INSTANCE_DB)
        viol_after = count_violations(driver, INSTANCE_DB)
        cost = cost_num_relabels(driver, INSTANCE_DB, tag="apoc_greedy")

        metrics = evaluate_label_recovery(gt_labels, before_labels, after_labels)

        row = {
            "algo": "apoc_greedy",
            "k": k,
            "seed": SEED,
            "violations_before": viol_before,
            "violations_after": viol_after,
            "repair_cost_num_relabels": cost,
            **metrics
        }
        rows.append(row)

        print(f"done k={k} | before={viol_before} after={viol_after} cost={cost} f1={metrics['f1']:.3f}")

df_results = pd.DataFrame(rows).sort_values(["k"]).reset_index(drop=True)

df_results["violations_removed"] = df_results["violations_before"] - df_results["violations_after"]
df_results["cost_per_violation_removed"] = df_results.apply(
    lambda r: (r["repair_cost_num_relabels"] / r["violations_removed"]) if r["violations_removed"] > 0 else None,
    axis=1
)

df_results


Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information